In [158]:
#DATA
import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.
import pandas as pd
import numpy as np
#random forest for regressor
from sklearn.ensemble import RandomForestRegressor
#split features  and target
from sklearn.model_selection import train_test_split
#Pour regression au lieu de accuracy c'est mean_square_error
from sklearn.metrics import mean_squared_error
#Ne pas afficher le warning lors du fit par exemple
#Import for cross_validation
from sklearn.model_selection  import cross_val_score
#import random forest for regression
from sklearn.ensemble import RandomForestRegressor
#Validation
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

['train.csv', 'sample_submission.csv', 'test.csv']


## IMPORT TRAIN - TEST - SUBMISSION

In [ ]:
# Train File
X_train = pd.read_csv('../input/train.csv', index_col='id')
#Test File
X_test = pd.read_csv('../input/test.csv', index_col='id')
#Submission File
submission = pd.read_csv('../input/sample_submission.csv')

## EDA

In [152]:
X_train.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
id,,,,,,,,,,
id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


### ENCODING

In [127]:
#a = pd.get_dumnies(X_train['store_and_fwd_flag'], prefix='store_and_fwd_flag', drop_first=True)  
#X_train[dumnies.columns] = dummies
# TRAIN
X_train.loc[X_train['store_and_fwd_flag'] == 'N', 'store_and_fwd_flag'] = 0
X_train.loc[X_train['store_and_fwd_flag'] == 'Y', 'store_and_fwd_flag'] = 1
# TEST
X_test.loc[X_test['store_and_fwd_flag'] == 'N', 'store_and_fwd_flag'] = 0
X_test.loc[X_test['store_and_fwd_flag'] == 'Y', 'store_and_fwd_flag'] = 1

In [128]:
#X_train.head(1)
#X_train = X_train.sort_values(by='trip_duration', ascending=False)

In [155]:
X_train = X_train[(X_train['trip_duration']  < 8000) & (X_train['passenger_count'] >= 1)]


#Gel only trip > 200s ie 3mn
#X_train = X_train[(X_train['trip_duration']  > 200) & (X_train['store_and_fwd_flag'] < 7500)]
#Delete line where trip_duration > 43200 ie 12h and store_and_fwd_flag = 1
#X_train = X_train.drop(X_train[(X_train['trip_duration']  > 43200) & (X_train['store_and_fwd_flag'] == 1)].index)
# Delete Trip < 10mn and store_and_fwd_flag = 0
#X_train = X_train.drop(X_train[(X_train['trip_duration']  <10 ) & (X_train['store_and_fwd_flag'] == 0)].index)
#X_train = X_train[(X_train['trip_duration']  < 43200) & (X_train['store_and_fwd_flag'] != 1 )]
#X_train.sort_values(by='trip_duration', ascending=True)

#X_train['store_and_fwd_flag'].value_counts()

In [157]:
#X_train['passenger_count'].value_counts()

1    1025592
2     208080
5      77350
3      59304
6      47850
4      28070
7          3
9          1
8          1
Name: passenger_count, dtype: int64

In [131]:
#X_train.describe()

In [132]:
X_test.head(1)

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
id,,,,,,,,
id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.75668,0


In [133]:
#X_test.describe()

## Function features and target

In [134]:
## Fonction features et target
def split_dataset(df, features, target='trip_duration'):
    X = df[features]
    y = df[target]
    return X, y

## Function split date

In [135]:
def date_split(df_train, df_test, date='pickup_datetime'):
    ##X_train
    cols=df_train[date]
    date_cols=pd.to_datetime(cols)
    df_train['year'] = date_cols.dt.year
    df_train['month'] = date_cols.dt.month
    df_train['day'] = date_cols.dt.day
    df_train['hour'] = date_cols.dt.hour
    df_train['minute'] = date_cols.dt.minute
    df_train['second'] = date_cols.dt.second
    #df_train = df_train.drop(['pickup_datetime'], axis=1)
    ##X_test
    cols2=df_test[date]
    date_cols2=pd.to_datetime(cols2)
    df_test['year'] = date_cols2.dt.year
    df_test['month'] = date_cols2.dt.month
    df_test['day'] = date_cols2.dt.day
    df_test['hour'] = date_cols2.dt.hour
    df_test['minute'] = date_cols2.dt.minute
    df_test['second'] = date_cols2.dt.second
    #df_test = df_test.drop(['pickup_datetime'], axis=1)
    return df_train, df_test

In [136]:
#X_train['pickup_datetime']
#date_cols=pd.to_datetime(cols)
#X_train['year'] = date_cols.dt.year
#X_train.head(1)

# TREATMENT

## Function same columns  X_train and X_test

In [137]:
#Split  date for X_train et X_test
X_train, X_test = date_split(X_train, X_test)

In [138]:
def Get_cols(df, features_test=X_test.columns):
    #get X_test columns in  X_train
    X_train_features = df[features_test]
    return  X_train_features


In [139]:
#Same columns for X_test and X_train
X_trainGet_cols = Get_cols(X_test)
#Get only columns numbers
numbers = X_trainGet_cols.select_dtypes(np.number)
#Definition features and target in the file train
X_train_features, y_train_target = split_dataset(X_train, features=numbers.columns)
#X_test numbers
X_test = X_test.select_dtypes(np.number)

##### OPTIMIZATION #########


##  Validation

In [140]:
#rf = RandomForestRegressor()
#kf = KFold(n_splits=5, random_state=1)                                                                                      
#loses = cross_val_score(rf, X_train_features, y_train_target, cv=kf, scoring='neg_mean_squared_log_error')
# np.sqrt(-loses.mean())
#loses = [np.sqrt(-l) for l in loses]
# np.mean(loses)
#loses[:5]

In [141]:
#Initialise Random Forest regressor
#rf = RandomForestRegressor()
#cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=42)
##Cross Validation
#lossess = -cross_val_score(rf, X_train_features, y_train_target, cv=cv, scoring='neg_mean_squared_log_error')

## FIT ALL THE TRAIN

In [142]:
## FIT all the train set
rf = RandomForestRegressor()
rf.fit(X_train_features, y_train_target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [143]:
#X_train_features.head()

In [144]:
#X_test.head()

## PREDICTION

In [145]:
#X_train_features.head(1)

In [146]:
##### Predict in the training
y_train_pred = rf.predict(X_train_features)

###### Predict in the test
y_test_pred = rf.predict(X_test)

y_test_pred.mean()

1039.4154059769585

# SUBMISSION

In [147]:
#!rm submission.csv

In [148]:
submission["trip_duration"] = y_test_pred
#Convertir notre fichier en csv
submission.to_csv('submission.csv', index=False)

__notebook_source__.ipynb  submission.csv
